# Model Evaluation & Selection

![](images/justice-icon.jpg)

## Objective

This module is going to introduce you...

1. TBD
2. TBD
3. TBD

## Quick refresher

But first, let's review a few things that we learned in the previous modules.

### Data prep

In [5]:
# packages used
import pandas as pd
from sklearn.model_selection import train_test_split

# import data
adult_census = pd.read_csv('../data/adult-census.csv')

# separate feature & target data
target = adult_census['class']
features = adult_census.drop(columns='class')

# drop the duplicated column `"education-num"` as stated in the data exploration notebook
features = features.drop(columns='education-num')

# split into train & test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=123)

### Feature engineering

In [6]:
# packages used
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# create selector object based on data type
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

# get columns of interest
numerical_columns = numerical_columns_selector(features)
categorical_columns = categorical_columns_selector(features)

# preprocessors to handle numeric and categorical features
numerical_preprocessor = StandardScaler()
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")

# transformer to associate each of these preprocessors with their respective columns
preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard_scaler', numerical_preprocessor, numerical_columns)
])

### Modeling

In [7]:
# packages used
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

# Pipeline object to chain together modeling processes
model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))
model

# fit our model
_ = model.fit(X_train, y_train)

# score on test set
model.score(X_test, y_test)

0.8503808041929408

## Resampling & cross-validation

In our "03-first_model.ipynb" notebook we split our data into training and testing sets and we assessed the performance of our model on the test set. Unfortunately, there are a few pitfalls to this approach:

1. If our dataset is small, a single test set may not provide realistic expectations of our model's performance on unseen data.
2. A single test set does not provide us any insight on variability of our model's performance.
3. Using our test set to drive our model building process can bias our results via _data leakage_.

Resampling methods provide an alternative approach by allowing us to repeatedly fit a model of interest to parts of the training data and test its performance on other parts of the training data.

![](images/resampling.svg)

<div class="admonition note alert alert-info">
    <p class="first admonition-title" style="font-weight: bold;"><b>Note</b></p>
<p class="last">This allows us to train and validate our model entirely on the training data and not touch the test data until we have selected a final "optimal" model.</p>
</div>

The two most commonly used resampling methods include ___k-fold cross-validation___ and ___bootstrap sampling___. This module focuses on using k-fold cross-validation.

## K-fold cross-validation

Cross-validation consists of repeating the procedure such that the training and testing sets are different each time. Generalization performance metrics are collected for each repetition and then aggregated. As a result we can get an estimate of the variability of the model’s generalization performance.

_k_-fold cross-validation (aka _k_-fold CV) is a resampling method that randomly divides the training data into _k_ groups (aka folds) of approximately equal size. 

![](images/cross_validation_diagram.png)

The model is fit on $k-1$ folds and then the remaining fold is used to compute model performance.  This procedure is repeated _k_ times; each time, a different fold is treated as the validation set. 

This process results in _k_ estimates of the generalization error (say $\epsilon_1, \epsilon_2, \dots, \epsilon_k$). Thus, the _k_-fold CV estimate is computed by averaging the _k_ test errors, providing us with an approximation of the error we might expect on unseen data.

![](images/cv.png)

In scikit-learn, the function [`cross_validate`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html) allows to do cross-validation and you need to pass it the model, the data, and the target. Since there exists several cross-validation strategies, cross_validate takes a parameter `cv` which defines the splitting strategy.

<div class="admonition tip alert alert-warning">
    <p class="first admonition-title" style="font-weight: bold;"><b>Tip</b></p>
    <p class="last">In practice, one typically uses k=5 or k=10. There is no formal rule as to the size of k; however, as k gets larger, the difference between the estimated performance and the true performance to be seen on the test set will decrease.</p>
</ul>
</div>

In [8]:
%%time
from sklearn.model_selection import cross_validate

cv_result = cross_validate(model, X_train, y_train, cv=5)
cv_result

CPU times: user 3.44 s, sys: 65.2 ms, total: 3.5 s
Wall time: 3.52 s


{'fit_time': array([0.66824007, 0.7457099 , 0.64340782, 0.67427135, 0.59037399]),
 'score_time': array([0.02877188, 0.02912116, 0.02842212, 0.02938485, 0.02747893]),
 'test_score': array([0.85191757, 0.84548185, 0.85790336, 0.85094185, 0.85558286])}

The output of cross_validate is a Python dictionary, which by default contains three entries: 

- `fit_time`: the time to train the model on the training data for each fold, 
- `score_time`: the time to predict with the model on the testing data for each fold, and 
- `test_score`: the default score on the testing data for each fold.

In [9]:
scores = cv_result["test_score"]
print("The mean cross-validation accuracy is: "
      f"{scores.mean():.3f} +/- {scores.std():.3f}")

The mean cross-validation accuracy is: 0.852 +/- 0.004


<div class="admonition tip alert alert-warning">
    <p class="first admonition-title" style="font-weight: bold;"><b>Your Turn</b></p>
<p class="last">
TBD
</p>
</div>

## Evaluation metrics

* Evaluation metrics allow us to measure the predictive accuracy of our model – the difference between the predicted value ($\hat{y}_i$) and the actual value ($y_i$).

* We often refer to evaluation metrics as ___loss functions___: $f(y_{i} - \hat{y}_i)$

* Scikit-Learn provides multiple ways to compute evaluation metrics and refers to this concept as [___scoring___](https://scikit-learn.org/stable/modules/model_evaluation.html). 
   1. Estimator scoring method
   2. Individual scoring functions
   3. Scoring parameters

### Estimator scoring method

Every estimator (regression/classification model) has a default scoring method

Most classifiers return the mean accuracy of the model on the supplied $X$ and $y$:

In [26]:
# toy data
from sklearn.datasets import load_breast_cancer
X_cancer, y_cancer = load_breast_cancer(return_X_y=True)

# fit model
clf = LogisticRegression(solver='liblinear').fit(X_cancer, y_cancer)

# score 
clf.score(X_cancer, y_cancer)

0.9595782073813708

While most regressors return the $R^2$ metric:

In [27]:
# toy data
from sklearn.datasets import load_boston
X_boston, y_boston = load_boston(return_X_y=True)

# fit model
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_boston, y_boston)

# score
reg.score(X_boston, y_boston)

0.7406426641094095

### Individual scoring functions

However, these default evaluation metrics are often not the metrics most suitable to the business problem.

There are many loss functions to choose from; each with unique characteristics that can be beneficial for certain problems.
   * Regression problems
      - Mean squared error (MSE)
      - Root mean squared error (RMSE)
      - Mean absolute error (MAE)
      - etc.
   * Classification problems
      - Area under the curve (AUC)
      - Cross-entropy (aka Log loss)
      - Precision
      - etc.

Scikit-Learn provides many [scoring functions](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) to choose from.

In [28]:
from sklearn import metrics

The functions take actual y values and predicted y values -- $f(y_{i} - \hat{y}_i)$

__Example regression metrics__:

In [36]:
y_pred = reg.predict(X_boston)

# Mean squared error
metrics.mean_squared_error(y_boston, y_pred)

21.894831181729202

In [37]:
# Mean absolute percentage error
metrics.mean_absolute_percentage_error(y_boston, y_pred)

0.1641729880648995

__Example classification metrics__:

In [40]:
y_pred = clf.predict(X_cancer)

# Area under the curve
metrics.roc_auc_score(y_cancer, y_pred)

0.9543760900586651

In [41]:
# F1 score
metrics.f1_score(y_cancer, y_pred)

0.968011126564673

In [39]:
# multiple metrics at once!
print(metrics.classification_report(y_cancer, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.93      0.95       212
           1       0.96      0.97      0.97       357

    accuracy                           0.96       569
   macro avg       0.96      0.95      0.96       569
weighted avg       0.96      0.96      0.96       569



### Scoring parameters

And since we prefer to use cross-validation procedures, scikit-learn has incorporated a `scoring` parameter.

Most evaluation metrics have a predefined text string that can be supplied as a `scoring` argument. 

In [47]:
# say we wanted to use AUC as our loss function while using 5-fold validation
cross_validate(model, X_train, y_train, cv=5, scoring='roc_auc')

{'fit_time': array([0.78194714, 0.76312613, 0.71795416, 0.74163508, 0.67845583]),
 'score_time': array([0.03394103, 0.0323801 , 0.03217793, 0.03334308, 0.03055   ]),
 'test_score': array([0.90485472, 0.90326931, 0.91316978, 0.90553186, 0.90816178])}

<div class="admonition note alert alert-info">
    <p class="first admonition-title" style="font-weight: bold;"><b>Note</b></p>
    <p class="last">The unified scoring API in scikit-learn always <u>maximizes</u> the score, so metrics which need to be minimized are negated in order for the unified scoring API to work correctly. Consequently, some metrics such as <tt class="docutils literal">mean_squared_error()</tt> will use a predefined text string starting with <b>neg_</b> (i.e. 'neg_mean_squared_error' .</p>
</div>

In [45]:
# applying mean squared error in a regression k-fold cross validation procedure
cross_validate(reg, X_boston, y_boston, cv=5, scoring='neg_root_mean_squared_error')

{'fit_time': array([0.00466919, 0.00105381, 0.00070906, 0.00056982, 0.00099421]),
 'score_time': array([0.00061488, 0.00032616, 0.00024104, 0.00023293, 0.00043392]),
 'test_score': array([-3.52991509, -5.10378498, -5.75101191, -8.9867887 , -5.77179405])}

You can even supply [more than one metric](https://scikit-learn.org/stable/modules/model_evaluation.html#using-multiple-metric-evaluation) or even [define your own custom metric](https://scikit-learn.org/stable/modules/model_evaluation.html#defining-your-scoring-strategy-from-metric-functions).

In [49]:
# example of supplying more than one metric
metrics = ['accuracy', 'roc_auc']

cross_validate(model, X_train, y_train, cv=5, scoring=metrics)

{'fit_time': array([0.75666189, 0.74446177, 0.71956205, 0.73168302, 0.68257308]),
 'score_time': array([0.0659771 , 0.06203222, 0.0604248 , 0.06098986, 0.05946302]),
 'test_accuracy': array([0.85191757, 0.84548185, 0.85790336, 0.85094185, 0.85558286]),
 'test_roc_auc': array([0.90485472, 0.90326931, 0.91316978, 0.90553186, 0.90816178])}

<div class="admonition tip alert alert-warning">
    <p class="first admonition-title" style="font-weight: bold;"><b>Your Turn</b></p>
<p class="last">
TBD
</p>
</div>

## Hyperparameter tuning

TBD

<div class="admonition tip alert alert-warning">
    <p class="first admonition-title" style="font-weight: bold;"><b>Your Turn</b></p>
<p class="last">
TBD
</p>
</div>

## Wrapping up